In [10]:
import pandas as pd
import json

df = pd.read_json("anatole_data_train.json")
df_train=df.sample(frac=0.95,random_state=0)
df_eval=df.drop(df_train.index)

df_formatted = [
    {
        "messages": [
            {"role": "user", "content": row["instruction"]},
            {"role": "assistant", "content": row["output"]},
        ]
    }
    for index, row in df_train.iterrows()
]

df_formatted2 = [
    {
        "messages": [
            {"role": "user", "content": row["instruction"]},
            {"role": "assistant", "content": row["output"]},
        ]
    }
    for index, row in df_eval.iterrows()
]

with open("data_train.jsonl", "w") as f:
    for line in df_formatted:
        json.dump(line, f)
        f.write("\n")
with open("data_eval.jsonl", "w") as f:
    for line in df_formatted2:
        json.dump(line, f)
        f.write("\n")

In [29]:
len(df)

31077

In [13]:
import os
from mistralai.client import MistralClient
os.environ['MISTRAL_API_KEY']='ymjotzDIdJxKj611QnrKt70VEJsHtyGm'
api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

with open("data_train.jsonl", "rb") as f:
    data_train = client.files.create(file=("data_train.jsonl", f))
with open("data_eval.jsonl", "rb") as f:
    data_eval = client.files.create(file=("data_eval.jsonl", f))

In [ ]:
from mistralai.models.jobs import TrainingParameters

created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[data_train.id],
    validation_files=[data_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=10,
        learning_rate=0.0001,
        )
)
created_jobs

In [27]:
from mistralai.models.jobs import TrainingParameters
import asyncio

dry_run_job = client.jobs.create(
    model="open-mistral-7b",
    training_files=[data_train.id],
    validation_files=[data_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=150,
        learning_rate=0.0001,
    ),
    dry_run=True,
)
print(dry_run_job)

object='job.metadata' training_steps=50 train_tokens_per_step=131072 data_tokens=4455124 train_tokens=6553600 epochs=1.471 expected_duration_seconds=300


In [18]:
import asyncio